# Basics
Let's start with the basics by just interacting with the Google's Vertex AI Chat-Bison LLM. We will first install the required modules for this notebook and then use ChatModel to interact with it. We will also install python-dotenv so we can retrieve credentials from a .env file.

In [ ]:
%pip install google-cloud-aiplatform

Now we will import our modules. Authentication is handled via the gcloud init commands as mentioned in the setup_vertex.md file.

In [ ]:
import vertexai
from vertexai.language_models import ChatModel, InputOutputTextPair
import os
import IPython

vertexai.init() ## initalise the vertexai class

chat_model = ChatModel.from_pretrained("chat-bison")

parameters = {
    "temperature": 0.7,  # Temperature controls the degree of randomness in token selection.
}

Here we will pick the chat-bison LLM through the vertexai class. This expects:

 - The string of context (as opposed to list of prompts)
 - List of examples if available 
 - The temperature (0 = deterministic vs. + = increase randomness)

In [ ]:
memory = "\n".join([  ### Vertex doesnt use the same concept of sending a list like openai - instead, i am taking the list and joining it togather as one string seperated by a newline (\n)

])

chat = chat_model.start_chat(  ### We start our chat with the context
    context=memory
)

response = chat.send_message( ## Send a new message along with parameters
    message="my life is potato, my blood is potato, when i sleep i dream of", **parameters)

IPython.display.Markdown(response.text)

In [ ]:

memory = [
]


chat = chat_model.start_chat(  ### We start our chat with the context
    context="\n".join(memory),
)

response = chat.send_message( ## Send a new message along with parameters
    message="Hello, how are you?", **parameters)

IPython.display.Markdown(response.text)

Unlike OpenAI, Vertex doesnt use lists of prompts and roles. What we have instead is the context parameter which would be one string and the examples parameter which takes a list of examples contained within "InputOutputTextPair". Let's check it out below:

In [ ]:


memory = [
    "You only speak in pirate language.",
    "You hate talking about rainbows and will refuse to answer."
]

examples = [
    InputOutputTextPair(
        input_text='Hello, how are you?',
        output_text='Arr, me hearty! I be quite well, thank ye for asking.'
    )
]


chat = chat_model.start_chat(  ### We start our chat with the context
    context="\n".join(memory),
    examples=examples
)

response = chat.send_message( ## Send a new message along with parameters
    message="Where do rainbows come from?", **parameters)

IPython.display.Markdown(response.text)

But as you can see from the answer it gave, it doesnt perform well with instructions when compared with OpenAI...

In [ ]:

memory = [
    "You only speak in pirate language.",
    "You hate talking about rainbows and will refuse to answer."
]

examples = [
    InputOutputTextPair(
        input_text='Hello, how are you?',
        output_text='Arr, me hearty! I be quite well, thank ye for asking.'
    )
]


chat = chat_model.start_chat(  ### We start our chat with the context
    context="\n".join(memory),
    examples=examples
)

response = chat.send_message( ## Send a new message along with parameters
    message="Where do rainbows come from?", **parameters)

print('first answer: '+response.text)



# Let's append the bot response (it will say it hates maths, etc...) to the memory
bot_response = response.text

memory.append(
    bot_response
)


# Let us ask another time
response = chat.send_message( ## Send a new message along with parameters
    message="I thought you didnt like to talk about rainbows??", **parameters)



# YOU SEND THE MEMORY FOR EVERY NEW MESSAGE YOU SUBMIT. THE API TOKEN LIMITS APPLY TO THE WHOLE LIST OF PROMPTS - NOT THE ONE MESSAGE

print('second answer: '+response.text)




If you've been using ChatGPT or any other kind of chatbot, you will notice that the response is streamed back to you rather than you having to wait for the full response. Streaming the response works out much better, especially when you use LLM for long context tasks or coding. Let us see how this works! We will take the previous example and let the LLM stream back the response token by token in a Python generator.

Unlike lists and dictionaries which keep all the items in memory and then returns, a generator returns the items (in this case tokens) one by one until completion.

In [ ]:
memory = [
    "You only speak in pirate language.",
    "You hate talking about rainbows and will refuse to answer."
]

examples = [
    InputOutputTextPair(
        input_text='Hello, how are you?',
        output_text='Arr, me hearty! I be quite well, thank ye for asking.'
    )
]


chat = chat_model.start_chat(  ### We start our chat with the context
    context="\n".join(memory),
    examples=examples
)

responses = chat.send_message_streaming( ## Send a new message along with parameters
    message="Where do rainbows come from?", **parameters)

for response in responses:
    print(response.text)

Unlike OpenAI, Vertex's streamed response is faster because it includes multiple tokens at the same time. So there's no need to speed it up.